In [30]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import torch.nn as nn

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
from datasets import Dataset

df = pd.read_csv("/content/drive/MyDrive/Colab/IntelliGuard/All_Datasets.csv")
df = df.dropna(subset=["text", "label"])

print(len(df))

21082


In [33]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16865 [00:00<?, ? examples/s]

Map:   0%|          | 0/4217 [00:00<?, ? examples/s]

In [35]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [36]:
labels = np.array(train_df['label'])
unique_classes, class_counts = np.unique(labels, return_counts=True)
weights = 1.0 / class_counts
print("Class counts:", dict(zip(unique_classes, class_counts)))
print("Class weights:", weights)

Class counts: {np.int64(0): np.int64(11200), np.int64(1): np.int64(5665)}
Class weights: [8.92857143e-05 1.76522507e-04]


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(unique_classes)
).to(device)

class_weights = torch.tensor(weights, dtype=torch.float).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
def evaluate(model, dataloader, device):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average='binary')
    print(f"Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(true_labels, preds))
    print(classification_report(true_labels, preds, digits=4))
    return acc

In [39]:
from tqdm.notebook import tqdm

EPOCHS = 40
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{EPOCHS}, Average Loss: {total_loss/len(train_loader):.4f}")
    evaluate(model, test_loader, device)

Epoch 1/40:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 1/40, Average Loss: 0.4970
Accuracy: 0.8295, Precision: 0.7570, Recall: 0.7255, F1: 0.7409
Confusion Matrix:
[[2470  330]
 [ 389 1028]]
              precision    recall  f1-score   support

           0     0.8639    0.8821    0.8729      2800
           1     0.7570    0.7255    0.7409      1417

    accuracy                         0.8295      4217
   macro avg     0.8105    0.8038    0.8069      4217
weighted avg     0.8280    0.8295    0.8286      4217



Epoch 2/40:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 2/40, Average Loss: 0.3394
Accuracy: 0.8357, Precision: 0.7855, Recall: 0.7029, F1: 0.7419
Confusion Matrix:
[[2528  272]
 [ 421  996]]
              precision    recall  f1-score   support

           0     0.8572    0.9029    0.8795      2800
           1     0.7855    0.7029    0.7419      1417

    accuracy                         0.8357      4217
   macro avg     0.8214    0.8029    0.8107      4217
weighted avg     0.8331    0.8357    0.8332      4217



Epoch 3/40:   0%|          | 0/1055 [00:00<?, ?it/s]

...

Epoch 38/40, Average Loss: 0.0262
Accuracy: 0.8103, Precision: 0.7153, Recall: 0.7234, F1: 0.7193
Confusion Matrix:
[[2392  408]
 [ 392 1025]]
              precision    recall  f1-score   support

           0     0.8592    0.8543    0.8567      2800
           1     0.7153    0.7234    0.7193      1417

    accuracy                         0.8103      4217
   macro avg     0.7872    0.7888    0.7880      4217
weighted avg     0.8108    0.8103    0.8106      4217



Epoch 39/40:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 39/40, Average Loss: 0.0243
Accuracy: 0.8193, Precision: 0.7494, Recall: 0.6944, F1: 0.7209
Confusion Matrix:
[[2471  329]
 [ 433  984]]
              precision    recall  f1-score   support

           0     0.8509    0.8825    0.8664      2800
           1     0.7494    0.6944    0.7209      1417

    accuracy                         0.8193      4217
   macro avg     0.8002    0.7885    0.7936      4217
weighted avg     0.8168    0.8193    0.8175      4217



Epoch 40/40:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 40/40, Average Loss: 0.0243
Accuracy: 0.8226, Precision: 0.7472, Recall: 0.7135, F1: 0.7300
Confusion Matrix:
[[2458  342]
 [ 406 1011]]
              precision    recall  f1-score   support

           0     0.8582    0.8779    0.8679      2800
           1     0.7472    0.7135    0.7300      1417

    accuracy                         0.8226      4217
   macro avg     0.8027    0.7957    0.7990      4217
weighted avg     0.8209    0.8226    0.8216      4217



In [40]:
save_path = "/content/drive/MyDrive/Colab/IntelliGuard/bert_rumor"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved to {save_path}")

Model and tokenizer saved to /content/drive/MyDrive/Colab/IntelliGuard/bert_rumor


In [41]:
from transformers import pipeline
clf = pipeline("text-classification", model=save_path, tokenizer=save_path, device=0 if torch.cuda.is_available() else -1)
print(clf("This is a rumor tweet"))

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9990455508232117}]
